# Hamidreza Amirzadeh
## 401206999
### Note: This work was done under collaboration with Ali Abdollahi

# **Preface**
In this notebook you are going to implement a powerful blackbox attack namely [NES](https://arxiv.org/pdf/1804.08598.pdf) and test it on a regular target model, then you will use a simple but powerful defense called  Random Noise Defense ([RND](https://arxiv.org/pdf/2104.11470.pdf)) against this attack on the same target model in order to check the power of the aforementioned attack in the presence of this so-called defense.
You will have to use a **CIFAR10 ResNet18 model** as the target model for this set of experiments.
You may want to train a new model or load an already trained model, your choice, but this model must have at least 94% accuracy on CIFAR10 test set as told in the previous exercises.

P.S. Don't normalize the data used for training.


In [ ]:
from google.colab import drive
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from typing import Type
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
if device == 'cuda':
  print(torch.cuda.get_device_name(0))

drive.mount('/content/drive')
%cd /content/drive/MyDrive/

from resnet import *

batch_size = 128
transform = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/myCIFAR10', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/myCIFAR10', train = False, download = True, transform = transforms.Compose([transforms.ToTensor(),]))
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

cuda
Tesla T4
Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
model = ResNet18()
model = model.to(device)

In [ ]:
# load model
model_name = "resnet18_cifar10_model_hw5.pth"
CIFAR10_model_PATH = "/content/drive/MyDrive/" + model_name
state_dict = torch.load(CIFAR10_model_PATH)
model.load_state_dict(state_dict)

In [ ]:
# init to train model
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9 , weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

In [ ]:
epochs = 100
for epoch in range(epochs):
    model.train()
    print("epoch " + str(epoch))
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.3f}')
            running_loss = 0.0
    if scheduler:
        scheduler.step()
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Standard Accuracy of ResNet18 model on the 10000 test images: {100 * correct / total} %')



print('Finished Training')

epoch 0
[1,     2] loss: 4.704
Standard Accuracy of ResNet18 model on the 10000 test images: 53.16 %
epoch 1
[2,     2] loss: 1.880
Standard Accuracy of ResNet18 model on the 10000 test images: 68.94 %
epoch 2
[3,     2] loss: 1.578
Standard Accuracy of ResNet18 model on the 10000 test images: 72.71 %
epoch 3
[4,     2] loss: 1.244
Standard Accuracy of ResNet18 model on the 10000 test images: 69.43 %
epoch 4
[5,     2] loss: 1.070
Standard Accuracy of ResNet18 model on the 10000 test images: 80.93 %
epoch 5
[6,     2] loss: 0.987
Standard Accuracy of ResNet18 model on the 10000 test images: 78.96 %
epoch 6
[7,     2] loss: 0.966
Standard Accuracy of ResNet18 model on the 10000 test images: 81.97 %
epoch 7
[8,     2] loss: 0.992
Standard Accuracy of ResNet18 model on the 10000 test images: 80.36 %
epoch 8
[9,     2] loss: 0.593
Standard Accuracy of ResNet18 model on the 10000 test images: 83.77 %
epoch 9
[10,     2] loss: 0.697
Standard Accuracy of ResNet18 model on the 10000 test image

In [ ]:
epochs = 100
for epoch in range(epochs):
    model.train()
    print("epoch " + str(epoch))
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1:    # print every 2000 mini-batches #1999
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.3f}')
            running_loss = 0.0
    if scheduler:
        scheduler.step()
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Standard Accuracy of ResNet18 model on the 10000 test images: {100 * correct / total} %')



print('Finished Training')

epoch 0
[1,     2] loss: 0.044
Standard Accuracy of ResNet18 model on the 10000 test images: 90.48 %
epoch 1
[2,     2] loss: 0.048
Standard Accuracy of ResNet18 model on the 10000 test images: 91.14 %
epoch 2
[3,     2] loss: 0.071
Standard Accuracy of ResNet18 model on the 10000 test images: 90.95 %
epoch 3
[4,     2] loss: 0.043
Standard Accuracy of ResNet18 model on the 10000 test images: 91.95 %
epoch 4
[5,     2] loss: 0.042
Standard Accuracy of ResNet18 model on the 10000 test images: 92.39 %
epoch 5
[6,     2] loss: 0.056
Standard Accuracy of ResNet18 model on the 10000 test images: 89.7 %
epoch 6
[7,     2] loss: 0.097
Standard Accuracy of ResNet18 model on the 10000 test images: 92.22 %
epoch 7
[8,     2] loss: 0.031
Standard Accuracy of ResNet18 model on the 10000 test images: 92.26 %
epoch 8
[9,     2] loss: 0.113
Standard Accuracy of ResNet18 model on the 10000 test images: 91.99 %
epoch 9
[10,     2] loss: 0.089
Standard Accuracy of ResNet18 model on the 10000 test images

In [ ]:
# save model
model.eval()
model_name = "resnet18_cifar10_model_hw5.pth"
CIFAR10_model_PATH = "/content/drive/MyDrive/" + model_name
torch.save(model.state_dict(), CIFAR10_model_PATH)

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Standard Accuracy of ResNet18 model on the 10000 test images: {100 * correct / total} %')

Standard Accuracy of ResNet18 model on the 10000 test images: 94.86 %


# **Natural Evolutionary Strategies (NES)**

Firstly complete the code for the attack itself and then the add_Gaussian method used for the Random Noise Defense in the NES attack class.

In [ ]:
import torch
import numpy as np
import torch as ch
import torchvision
import torchvision.transforms.functional as F
import torchvision.transforms as transforms
import argparse
import json
import pdb
import os
import time
import math
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn import DataParallel
from torch.nn.modules import Upsample
from matplotlib import pyplot as plt

CLASSIFIERS = {
    "resnet18": (models.resnet50, 224),
}

NUM_CLASSES = {
    "cifar10": 10,
}

# Set the path to your dataset
CIFAR10_PATH = "/content/drive/MyDrive/myCIFAR10"

if CIFAR10_PATH == "":
  raise ValueError("Please fill out the path to Cifar10!")

class NES():

  def __init__(self, model_to_fool, args, dataset_size, RND_coefficient, device):


      if args != None:
        self.args = args
      else:
        self.max_queries = 10000
        self.fd_eta = 0.01
        self.image_lr = 0.01
        self.mode = "linf"
        self.json_config = None
        self.epsilon = 0.05
        self.batch_size = 10
        self.log_process = "store_true"
        self.gradient_iters = 15
        self.total_images = 10000
        self.classifier = 'resnet18'
        self.args = {'max_queries': self.max_queries,
          'fd_eta': self.fd_eta,
          'image_lr': self.image_lr,
          'mode': self.mode,
          'json_config': self.json_config,
          'epsilon': self.epsilon,
          'batch_size': self.batch_size,
          'log_progress': self.log_progress,
          'gradient_iters': self.gradient_iters,
          'total_images': self.total_images,
          'classifier': self.classifier,
          }
      self.model_to_fool = model_to_fool
      self.dataset_size = dataset_size
      self.RND_coefficient = RND_coefficient
      self.device = device
      if self.device == 'cuda':
        ch.set_default_tensor_type('torch.cuda.FloatTensor')
      else:
        ch.set_default_tensor_type('torch.FloatTensor')

      if self.dataset_size == 32:
        self.channel_size = 3
      else:
        raise ValueError("Please check out the dataset_size parameter...")


  """For setting up the Random Noise Defense"""

  def add_Gausian(self, input_image, coeff):
    # Firstly define a random epsilon with the input image shape
    # Remember that it should be a float
    # Add the epsilon with the proper coefficient to the input image
    # Don't forget to clamp it between the right values
    # P.S. Remember to convert everything to the right device
    #############################
    # Your code goes here
    epsilon = ch.randn_like(input_image)
    return torch.clamp(input_image + coeff*epsilon, 0, 1)
    #############################


  def generate(self):
      dataset = None
      if self.dataset_size == 32:
        transform = transforms.Compose([transforms.ToTensor(), transforms.Resize(self.dataset_size), transforms.CenterCrop(self.dataset_size),])
        dataset = torchvision.datasets.CIFAR10(root = CIFAR10_PATH, train = False, download = False, transform = transform)
      else:
        raise ValueError("Please check out the dataset_size parameter...")

      dataset_loader = DataLoader(dataset, batch_size=self.args["batch_size"])
      total_correct, total_adv, total_queries = 0, 0, 0
      for i, (images, targets) in enumerate(dataset_loader):
          if i*self.args["batch_size"] >= self.args["total_images"]:
              break

          res = self.make_adversarial_examples(images.to(self.device), targets.to(self.device), self.args, self.model_to_fool, self.dataset_size)
          ncc = res['num_correctly_classified'] # Number of correctly classified images (originally)
          num_adv = ncc * res['success_rate'] # Success rate was calculated as (# adv)/(# correct classified)
          queries = num_adv * res['average_queries'] # Average queries was calculated as (total queries for advs)/(# advs)
          total_correct += ncc
          total_adv += num_adv
          total_queries += queries
          print("i: " + str(i))
          sucRate = res['success_rate']
          print(f'success_rate : {sucRate}')
          print(f'number of Adversarials : {int(num_adv)}')
          print(f'number of queries : {int(queries)}')

      print("-"*80)
      if total_adv != 0:
        print("Final Success Rate: {succ} | Final Average Queries for Successful Adv Examples: {aq}".format(
                aq=int(total_queries/total_adv),
                succ=total_adv/total_correct))
      else:
        """If total adversarial examples is zero then we won't print any avg number of queries for success
        because we had no success and the avg number of queries has exceeded the budget"""
        print("Final Success Rate: {succ} ".format(
                # aq = total_queries,
                succ=total_adv/total_correct))
      print("-"*80)
      return


  def norm(self, t):
      assert len(t.shape) == 4
      norm_vec = ch.sqrt(t.pow(2).sum(dim=[1,2,3])).view(-1, 1, 1, 1)
      norm_vec += (norm_vec == 0).float()*1e-8
      return norm_vec

  ###
  # Different optimization steps
  # All take the form of func(x, g, lr)
  # eg: exponentiated gradients
  # gd: gradient descent
  # l2/linf: projected gradient descent
  ###


  def eg_step(self, x, g, lr):
      real_x = (x + 1)/2 # from [-1, 1] to [0, 1]
      pos = real_x*ch.exp(lr*g)
      neg = (1-real_x)*ch.exp(-lr*g)
      new_x = pos/(pos+neg)
      return new_x*2-1

  def linf_step(self, x, g, lr):
      return x + lr*ch.sign(g)

  def l2_prior_step(self, x, g, lr):
      new_x = x + lr*g/self.norm(g)
      norm_new_x = self.norm(new_x)
      norm_mask = (norm_new_x < 1.0).float()
      return new_x*norm_mask + (1-norm_mask)*new_x/norm_new_x

  def gd_prior_step(self, x, g, lr):
      return x + lr*g

  def l2_image_step(self, x, g, lr):
      return x + lr*g/self.norm(g)

  ##
  # Projection steps for l2 and linf constraints:
  # All take the form of func(new_x, old_x, epsilon)
  ##

  def l2_proj(self, image, eps):
      orig = image.clone()
      def proj(new_x):
          delta = new_x - orig
          out_of_bounds_mask = (self.norm(delta) > eps).float()
          x = (orig + eps*delta/self.norm(delta))*out_of_bounds_mask
          x += new_x*(1-out_of_bounds_mask)
          return x
      return proj

  def linf_proj(self, image, eps):
      orig = image.clone()
      def proj(new_x):
          return orig + ch.clamp(new_x - orig, -eps, eps)
      return proj

  ##
  # Main functions
  ##

  def make_adversarial_examples(self, image, true_label, args, model_to_fool, DATASET_SIZE):
      '''
      The main process for generating adversarial examples with priors.
      '''
      # Initial setup
      prior_size = DATASET_SIZE
      upsampler = Upsample(size=(DATASET_SIZE, DATASET_SIZE))
      total_queries = ch.zeros(args["batch_size"])
      prior = ch.zeros(args["batch_size"], self.channel_size, prior_size, prior_size)
      dim = prior.nelement()/args["batch_size"]
      image_step = self.l2_image_step if args["mode"] == 'l2' else self.linf_step
      proj_maker = self.l2_proj if args["mode"] == 'l2' else self.linf_proj
      proj_step = proj_maker(image, args["epsilon"])

      # Loss function
      criterion = ch.nn.CrossEntropyLoss(reduction='none')

      def normalized_eval(x):
          x_copy = x.clone()

          # You can comment or uncomment the code needed for a specific case
          """################## Normal prediction ##################"""
          x_copy = ch.clamp(x_copy, 0, 1)
          return self.model_to_fool(x_copy)

          """################## Prediction using random noise defense ##################"""
          x_copy_noisy = self.add_Gausian(x_copy, self.RND_coefficient)
          x_copy_noisy = ch.clamp(x_copy_noisy, 0, 1)
          return self.model_to_fool(x_copy_noisy)

      L = lambda x: criterion(normalized_eval(x), true_label)

      losses = L(image)

      # Original classifications
      orig_images = image.clone()

      # You can comment or uncomment the code needed for a specific case
      """##################Normal prediction##################"""
      orig_classes = self.model_to_fool(orig_images).argmax(1).to(self.device)

      """################## Prediction using random noise defense ##################"""
      image_noisy = self.add_Gausian(image, self.RND_coefficient)
      image_noisy = ch.clamp(image_noisy, 0, 1)
      orig_classes = self.model_to_fool(image_noisy).argmax(1).to(self.device)

      correct_classified_mask = (orig_classes == true_label).float()
      total_ims = correct_classified_mask.sum()
      not_dones_mask = correct_classified_mask.clone()

      t = 0
      while not ch.any(total_queries > args["max_queries"]):
          t += args["gradient_iters"]*2
          if t >= args["max_queries"]:
              break

          prior = ch.zeros_like(image)
          for _ in range(args["gradient_iters"]):

              # Define a tensor of random Gaussian noise with the input image shape
              # Then devide this tensor by (dim**0.5)
              # Now add and subtract the defined noise with the fd_eta coefficient with the original image
              # Finally build the gradient estimation of the loss using the finite difference method
              #############################
              # Your code goes here
              exp_noise = ch.randn_like(image)/(dim**0.5)
              est_deriv = (L(image + args['fd_eta']*exp_noise) - L(image - args['fd_eta']*exp_noise))/args['fd_eta']
              #############################

              prior += est_deriv.view(-1, 1, 1, 1)*exp_noise

          # Preserve images that are already done,
          # Unless we are specifically measuring gradient estimation
          prior = prior*not_dones_mask.view(-1, 1, 1, 1)

          ## Update the image:
          # take a pgd step using the prior
          new_im = image_step(image, upsampler(prior*correct_classified_mask.view(-1, 1, 1, 1)), args["image_lr"])
          image = proj_step(new_im)
          image = ch.clamp(image, 0, 1)

          if args["mode"] == 'l2':
              if not ch.all(self.norm(image - orig_images) <= args["epsilon"] + 1e-3):
                  pass
                  # pdb.set_trace()
          else:
              if not (image - orig_images).max() <= args["epsilon"] + 1e-3:
                pass
                  # pdb.set_trace()

          ## Continue query count
          total_queries += 2*args["gradient_iters"]*not_dones_mask
          not_dones_mask = not_dones_mask*((normalized_eval(image).argmax(1) == true_label).float())

          ## Logging and stuff

          # new_losses = L(image)
          success_mask = (1 - not_dones_mask)*correct_classified_mask
          num_success = success_mask.sum()
          if num_success != 0:
            success_queries = ((success_mask*total_queries).sum()/num_success).cpu().item()
          else:
            success_queries = ((success_mask*total_queries).sum()).cpu().item()
          """If num_success == 0 it means that the number of successful adv examples was zero (No adv examples could be made!)
          so success queries must not be printed """

          current_success_rate = (num_success/correct_classified_mask.sum()).cpu().item()
          #not_done_loss = ((new_losses*not_dones_mask).sum()/not_dones_mask.sum()).cpu().item()
          max_curr_queries = total_queries.max().cpu().item()
          # if args["log_progress"]:
          #   print("Queries: %d (%d) | Success rate: %f | Average queries: %f" % (max_curr_queries , t, current_success_rate, success_queries))

          if current_success_rate == 1.0:
              break

      return {
              'average_queries': success_queries,
              'num_correctly_classified': correct_classified_mask.sum().cpu().item(),
              'success_rate': current_success_rate,
              'images_orig': orig_images.cpu().numpy(),
              'images_adv': image.cpu().numpy(),
              'all_queries': total_queries.cpu().numpy(),
              'correctly_classified': correct_classified_mask.cpu().numpy(),
              'success': success_mask.cpu().numpy()
      }

# **Attack Arguments Setup**

**Write a report and indicate the success rate and average number of qureies for sections A and B.**

A) Now you must launch the attack using different values;
For the whole CIFAR10 test set, first on the regular target model and without the Random Noise Defense, with fd_eta parameter of 0.01 and 0.001, using "linf" mode (2 results).

B) Then apply the Random Noise Defense on the target model with RND_coefficient of 0.01 and 0.02 against NES with fd_eta parameter of 0.01 and 0.001 again using the "linf" mode (4 results).

Analyse and compare the results of both sections in your report.


In [ ]:
budget = 1000
target_model_arch = 'resnet18'
dataset_size = 32

args = {'max_queries': budget,
              'fd_eta': 0.01,
              'image_lr': 0.01,
              'mode': 'linf',
              'json_config': None,
              'epsilon': 0.05,
              'batch_size': 100,
              'log_progress': "store_true",
              'gradient_iters': 15,
              'total_images': 10000,
              'classifier': target_model_arch,
              }
"""
RND_coefficient = ?
device = ?
target_model.eval()
"""
print('without defense :')

with torch.no_grad():
    # Launch the attack using the NES class and its generate method
    #############################
    # Your code goes here
    nes = NES(model_to_fool = model, dataset_size = dataset_size, args = args, device = device, RND_coefficient=0)
    model.eval()
    for fd_eta in [0.01, 0.001]:
      print(f'fd_eta = {fd_eta}')
      args['fd_eta'] = fd_eta
      _ = nes.generate()

    #############################

without defense :
fd_eta = 0.01


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


i: 0
success_rate : 0.9166666865348816
numAdv : 88
queries : 22679
i: 1
success_rate : 0.957446813583374
numAdv : 90
queries : 24210
i: 2
success_rate : 0.9270833134651184
numAdv : 88
queries : 21779
i: 3
success_rate : 0.9166666865348816
numAdv : 88
queries : 21539
i: 4
success_rate : 0.9684210419654846
numAdv : 91
queries : 23459
i: 5
success_rate : 0.9175257682800293
numAdv : 88
queries : 27600
i: 6
success_rate : 0.9375
numAdv : 90
queries : 23520
i: 7
success_rate : 0.9677419066429138
numAdv : 89
queries : 22709
i: 8
success_rate : 0.9347826242446899
numAdv : 86
queries : 25500
i: 9
success_rate : 0.8297872543334961
numAdv : 78
queries : 19320
i: 10
success_rate : 0.936170220375061
numAdv : 88
queries : 21780
i: 11
success_rate : 0.914893627166748
numAdv : 86
queries : 22470
i: 12
success_rate : 0.9270833134651184
numAdv : 88
queries : 24869
i: 13
success_rate : 0.8958333134651184
numAdv : 85
queries : 23189
i: 14
success_rate : 0.8969072103500366
numAdv : 86
queries : 23309
i: 15

In [ ]:
print('defence with RND_coefficient = 0.01 :')

with torch.no_grad():
    # Launch the attack using the NES class and its generate method
    #############################
    # Your code goes here
    nes = NES(model_to_fool = model, dataset_size = dataset_size, args = args, device = device, RND_coefficient=0.01)
    model.eval()
    for fd_eta in [0.01, 0.001]:
      print(f'fd_eta = {fd_eta}')
      args['fd_eta'] = fd_eta
      _ = nes.generate()

    #############################

defence with RND_coefficient = 0.01 :
fd_eta = 0.01
i: 0
success_rate : 0.27173912525177
numAdv : 24
queries : 11759
i: 1
success_rate : 0.301075279712677
numAdv : 28
queries : 9510
i: 2
success_rate : 0.2886597812175751
numAdv : 27
queries : 11609
i: 3
success_rate : 0.24742268025875092
numAdv : 23
queries : 6779
i: 4
success_rate : 0.28421053290367126
numAdv : 27
queries : 9960
i: 5
success_rate : 0.1875
numAdv : 18
queries : 7020
i: 6
success_rate : 0.25
numAdv : 24
queries : 7980
i: 7
success_rate : 0.29347825050354004
numAdv : 26
queries : 5789
i: 8
success_rate : 0.24468085169792175
numAdv : 23
queries : 7769
i: 9
success_rate : 0.17391304671764374
numAdv : 16
queries : 3390
i: 10
success_rate : 0.34736841917037964
numAdv : 32
queries : 10080
i: 11
success_rate : 0.23404255509376526
numAdv : 22
queries : 7079
i: 12
success_rate : 0.2210526317358017
numAdv : 21
queries : 6690
i: 13
success_rate : 0.1979166716337204
numAdv : 19
queries : 5100
i: 14
success_rate : 0.1632653027772903

In [ ]:
print('defence with RND_coefficient = 0.02 :')

with torch.no_grad():
    # Launch the attack using the NES class and its generate method
    #############################
    # Your code goes here
    nes = NES(model_to_fool = model, dataset_size = dataset_size, args = args, device = device, RND_coefficient=0.02)
    model.eval()
    for fd_eta in [0.01, 0.001]:
      print(f'fd_eta = {fd_eta}')
      args['fd_eta'] = fd_eta
      _ = nes.generate()

    #############################

defence with RND_coefficient = 0.02 :
fd_eta = 0.01
i: 0
success_rate : 0.3655914068222046
numAdv : 34
queries : 12840
i: 1
success_rate : 0.3181818127632141
numAdv : 27
queries : 8640
i: 2
success_rate : 0.31182795763015747
numAdv : 29
queries : 8730
i: 3
success_rate : 0.31182795763015747
numAdv : 29
queries : 9359
i: 4
success_rate : 0.3186813294887543
numAdv : 29
queries : 7590
i: 5
success_rate : 0.2395833283662796
numAdv : 22
queries : 7109
i: 6
success_rate : 0.35483869910240173
numAdv : 32
queries : 11309
i: 7
success_rate : 0.3222222328186035
numAdv : 29
queries : 8250
i: 8
success_rate : 0.31460675597190857
numAdv : 28
queries : 10560
i: 9
success_rate : 0.26966291666030884
numAdv : 23
queries : 8519
i: 10
success_rate : 0.3186813294887543
numAdv : 29
queries : 8730
i: 11
success_rate : 0.32608696818351746
numAdv : 30
queries : 7410
i: 12
success_rate : 0.2527472674846649
numAdv : 23
queries : 5010
i: 13
success_rate : 0.20879121124744415
numAdv : 19
queries : 4620
i: 14
succ

In [ ]:
print('defence with RND_coefficient = 0.02 :')

with torch.no_grad():
    # Launch the attack using the NES class and its generate method
    #############################
    # Your code goes here
    model.eval()
    for fd_eta in [0.001]:
      print(f'fd_eta = {fd_eta}')
      args['fd_eta'] =fd_eta
      NES(model_to_fool = model, dataset_size = dataset_size, args = args, device = device, RND_coefficient=0.02).generate()

    #############################

defence with RND_coefficient = 0.02 :
fd_eta = 0.001


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


i: 0
success_rate : 0.35483869910240173
numAdv : 32
queries : 11400
i: 1
success_rate : 0.3444444537162781
numAdv : 31
queries : 10590
i: 2
success_rate : 0.301075279712677
numAdv : 28
queries : 8670
i: 3
success_rate : 0.2777777910232544
numAdv : 25
queries : 8010
i: 4
success_rate : 0.3586956560611725
numAdv : 33
queries : 7710
i: 5
success_rate : 0.20000000298023224
numAdv : 19
queries : 5400
i: 6
success_rate : 0.30434781312942505
numAdv : 27
queries : 10349
i: 7
success_rate : 0.2888889014720917
numAdv : 26
queries : 5820
i: 8
success_rate : 0.2888889014720917
numAdv : 26
queries : 8370
i: 9
success_rate : 0.19540229439735413
numAdv : 16
queries : 5220
i: 10
success_rate : 0.3181818127632141
numAdv : 27
queries : 10110
i: 11
success_rate : 0.30434781312942505
numAdv : 27
queries : 9149
i: 12
success_rate : 0.260869562625885
numAdv : 23
queries : 7889
i: 13
success_rate : 0.24731183052062988
numAdv : 23
queries : 5970
i: 14
success_rate : 0.2857142984867096
numAdv : 28
queries : 10

In [ ]:
# Final results report


# without defense, fd_eta=0.01  : Final Success Rate: 0.93 | Final Average Queries for Successful Adv Examples: 264
# without defense, fd_eta=0.001 : Final Success Rate: 0.92 | Final Average Queries for Successful Adv Examples: 267

# defense with RND=0.01, fd_eta=0.01   : Final Success Rate: 0.25 | Final Average Queries for Successful Adv Examples: 334
# defense with RND=0.01, fd_eta=0.001  : Final Success Rate: 0.24 | Final Average Queries for Successful Adv Examples: 333

# defense with RND=0.02, fd_eta=0.01   : Final Success Rate: 0.31 | Final Average Queries for Successful Adv Examples: 316
# defense with RND=0.02, fd_eta=0.001  : Final Success Rate: 0.31 | Final Average Queries for Successful Adv Examples: 309